# Добро пожаловать!

Этот блокнот предназначен для парсинга и обработки информации с сайта **[dom.mingkh.ru](https://dom.mingkh.ru/)**. 

Сбор проводится **по отдельным городам** (не по целым регионам).

Впоследствии подготовленные данные геокодировать (= присвоить координаты) адреса двумя вариантами:

* с помощью API МинЖКХ;
* с помощью встроенной функции «Пакетный геокодер Nominatim» в QGIS. Проверить корректность написания адреса можно на сайте [Nominatim OpenStreetMap](https://nominatim.openstreetmap.org/ui/search.html).

**Внимание!** В базу МинЖКХ занесены далеко не все здания. Особенно редко учитываются частные постройки.

**Содержание блокнота**:
1. Импортируем библиотеки.
2. Инициализируем функцию.
3. Узнаём число домов в городе или регионе.
4. Собираем данные.
5. Дособираем данные.
6. Указываем координаты с помощью API.
7. Подготавливаем адреса для геокодирования в QGIS.
8. Скачиваем получившиеся данные.

**Москва** как субъект федерации ≠ Москва в базе МинЖКХ. Эксклавы и округа Новой Москвы указаны отдельно. Потому в url для старой Москвы название региона и название самого города дублируется: https://dom.mingkh.ru/moskva/moskva/houses

Относитесь к данным **скептически** в хорошем смысле слова, не доверяйте им на 100 %. Так, в Москве вы можете найти дом 1703 года на Аминьевском шоссе — очевидно, что это просто чья-то опечатка.

# Шаг 1. Импортируем библиотеки

In [ ]:
# Шаг 1. Импортируем нужные библиотеки

import time
import requests
import winsound
import numpy as np
import pandas as pd
from tqdm import tqdm
from random import randint
from bs4 import BeautifulSoup as soup

# Шаг 2. Инициализируем функцию

Нам нужна функция, которая пройдёт по веб-таблице со всеми жилыми домами города N, соберёт ссылки на страницы с детальной информацией про дома и вернёт датафрейм с подробной информацией.

В приведённой ниже функции мы собираем:

    1. Номер (порядковый в системе dom.mingkh для конкретного населённого пункта). 
    2. Адрес (дома).
    3. Ссылка (на детальное описание дома). 
    4. Площадь всего (жилые, нежилые помещения и общее имущество). 
    5. Год постройки (дома).
    6. Число этажей.
    7. Аварийный (ли дом, ответы «да» или «нет»).
    8. Состояние дома (исправный или аварийный).
    9. Количество квартир (в доме).
    10. Тип дома («многоквартирный дом» или нечто иное).
    11. Площадь жилых (помещений) в м2.
    12. Площадь нежилых (помещений) в м2. 
    13. Культурное наследие (является ли дом объектом культурного наследия).
    
Однако на страницах домов содержится больше информации — вы можете добавить её вручную, самостоятельно поработав с кодом по нашему примеру.

Примеры страниц для сбора: 

* [таблица всех жилых домов Москвы](https://dom.mingkh.ru/moskva/moskva/houses), 
* [подробные данные о конкретном доме](https://dom.mingkh.ru/moskva/moskva/716860).

**Внимание!** Чтобы код не сломался, по умолчанию числовые значения сохраняются как текст. Бывает, что в исходный данных вместо года указан прочерк —, например. Разные города, по всей видимости, заполнялись разными людьми, и предугадать все варианты сложно. Потому после сбора данных и перед этапом анализа всё равно необходима очистка.

In [ ]:
# Шаг 2. Инициализируем нужные функции

def get_a_df_from_one_page (initial_table):
    
    ''' Эта функция принимает на вход html-таблицу и возвращает pandas-датафрейм со собранными атрибутами. 
    Если страница дома получена успешно, но данных нет, в таблицу добавится значение "Нет информации". 
    Если же код по какой-то причине сработает с ошибкой, будут выведены сообщения "Nothing to show" и "No data".
    Первая строка каждой таблицы — это заголовки столбцов, поэтому по одной ошибке на страницу — это нормально.'''
    
    all_rows = []
    for every_row in initial_table.find_all ('tr'):
        all_elements = every_row.find_all('td')
        try:
            number = all_elements [0].text
            address = all_elements [2].text
            area = all_elements [3].text
            year = all_elements [4].text
            levels = all_elements [5].text
        except:
            print ('Nothing to add')
        
        try:
            link = 'https://dom.mingkh.ru' + every_row.find('a')['href']
            house_id = int(link.split('/')[-1])
            
            house = soup (requests.get(link).content)

            all_in_table_left = house.find_all ('table')[0].find_all ('td')
            all_in_table_left = [i.text for i in all_in_table_left if i.text != ' ']
            try:
                needs_repair = all_in_table_left[all_in_table_left.index('Дом признан аварийным ')+1]
            except:
                needs_repair = 'Нет информации'
            try:
                raggedness = all_in_table_left [all_in_table_left.index('Состояние дома ')+1]
            except:
                raggedness = 'Нет информации'
            try:
                flat_number = all_in_table_left [all_in_table_left.index('Количество квартир ')+1]
            except:
                flat_number = 'Нет информации'


            all_in_table_right = house.find_all ('table')[1].find_all ('td')
            all_in_table_right = [i.text for i in all_in_table_right if i.text != ' ']
            try:
                house_type = all_in_table_left [all_in_table_left.index('Тип дома ')+1]
            except:
                house_type = 'Нет информации'
                try:
                    house_type = all_in_table_right [all_in_table_right.index('Тип дома ')+1]
                except:
                    house_type = 'Нет информации'
            try:
                living_area = all_in_table_right [all_in_table_right.index('Площадь жилых помещений м2 ')+1]
            except:
                living_area = 'Нет информации'
            try:
                non_living_area = all_in_table_right [all_in_table_right.index('Площадь нежилых помещений м2 ')+1]
            except:
                non_living_area = 'Нет информации'
            try:
                culture = all_in_table_right [all_in_table_right.index('Статус объекта культурного наследия ')+1]
            except:
                culture = 'Нет информации'

            needed_in_table = [needs_repair, raggedness, flat_number, house_type, living_area, non_living_area, culture]

        except:
            print (every_row)
            print ('Nothing to show')
            needed_in_table = ['Нет информации', 'Нет информации', 'Нет информации', 
                               'Нет информации', 'Нет информации', 'Нет информации', 'Нет информации']

        try:
            all_rows_data = [number, address, link, house_id, area, year, levels]
            all_rows_data.extend (needed_in_table)
            all_rows.append (all_rows_data)
        except:
            print ('No data')
    first_page = pd.DataFrame (all_rows, columns = ['Номер', 'Адрес', 'Ссылка', 'ID дома', 'Площадь всего', 'Год постройки', 'Число этажей', 
                                       'Аварийный', 'Состояние дома', 'Количество квартир', 'Тип дома', 
                                       'Площадь жилых в м2', 'Площадь нежилых в м2', 'Культурное наследие'])
    return first_page

# Шаг 3. Узнаём число домов

Запустите ячейку. В выпавшее поле вставьте ссылку на город в формате https://dom.mingkh.ru/bashkortostan/ufa/houses или https://dom.mingkh.ru/moskva/moskva/houses. 

**Проверьте ссылку**:
1. В ссылке должен быть указан не только регион, но и непосредственно **город**. 
2. Ссылка должна оканчиваться на **слово houses** — иначе число страниц будет рассчитано неверно.

В строчке под ячейкой вы увидите, сколько в этом регионе домов и сколько страниц придётся собрать.

**Обязательно** запустите эту ячейку, она нужна для выполнения последующих ячеек. Перезапускайте её каждый раз, когда нужно поменять регион или город и собрать данные по новой ссылке.

In [ ]:
# Шаг 3. Узнаём, как много придётся собрать

def last_page_number (og_list):
    ''' Принимает на вход список ссылок в html-тэгах, собранных через soup.find_all.
    Возвращает номер последней страницы как int'''
    k = [int(i['data-ci-pagination-page']) for i in og_list if i.text == 'Последняя'][0]
    return k

base_url = input ('Вставьте ссылку на нужный город или регион: ')
region = base_url.split ('/')[3]
try:
    city = base_url.split ('/')[4]
except:
    print ('Выбран целый регион, без города')
base_x = soup (requests.get (base_url).content)
try:
    last_page = last_page_number (base_x.find_all ('a'))
    last_page_html = soup (requests.get (base_url+'?page='+str(last_page)).content)
    house_num = int(last_page_html.find_all ('td')[-6].text)
except IndexError:
    last_page = 1
    house_num = int(base_x.find_all ('td')[-6].text)
print (f'Выборка города {city} региона {region} состоит из {last_page} страниц и {house_num} домов.')

# Шаг 4. Собираем данные

Запускаем код для сбора данных. Обратите внимание, что если ваш компьютер заснёт, код прервёт выполнение. Стоит сохранить накопленный результат в отдельную переменную или файл, продолжить выполнение кода с достигнутой точки, а затем объединить получившиеся файлы.

Для ориентира: сбор данных по Москве (32 727 домов) занимает 3 часа.

Когда парсинг заканчивается, раздаётся звук.

In [ ]:
# Шаг 3. Код проходит по всем страницам веб-таблицы города и собирает данные в единую таблицу

general_df = pd.DataFrame ()

for i in tqdm (range (1, last_page+1)): # Правым краем диапазона ставим число страниц в таблице города + 1
    url = f'{base_url}?page={i}' 
    x = soup (requests.get (url).content)
    table = x.find ('table')
    result = get_a_df_from_one_page (table)
    general_df = general_df.append (result, ignore_index=True)
    time.sleep (randint (0,2))
winsound.Beep (440, 1000)

Запустите ячейку 3.0, чтобы проверить, всё ли в порядке

In [ ]:
# Шаг 3.0 Предпросмотр получившегося датафрейма 

general_df

Если всё хорошо, можно просто сохранить получившийся датафрейм в переменную, которая будет использоваться далее.

In [ ]:
# Шаг 3.1. Объединяем результаты в один датафрейм

final_df = general_df

Если код досрочно прервал работу, сохраните результаты во временный файл. Переименуйте его, чтобы не перепутать с другими.

In [ ]:
# Шаг 3.2 Сохранение временных результатов на компьютер

general_df.to_csv ('Временный.csv')

В моём случае при сборе данных Москвы выполнение кода прерывалось дважды, поэтому мне пришлось объединить три файла.

In [ ]:
# Шаг 3.3. Объединяем результаты в один датафрейм

df1 = pd.read_csv ('Дома 0_3150.csv').drop ('Unnamed: 0', axis=1)
df2 = pd.read_csv ('Дома 3001_28100.csv').drop ('Unnamed: 0', axis=1)
df3 = pd.read_csv ('Дома 28100_32727.csv').drop ('Unnamed: 0', axis=1)

final_df = df1.append (df2)
final_df = final_df.append (df3)
final_df = final_df.drop_duplicates()
final_df

# Шаг 5. Дособираем данные

У некоторых домов было очень мало информации, поэтому код вернул «Нет информации», хотя она просто была организована иным образом. Также «Нет информации» возвращалось в результате прервавшегося выполнения кода. Чтобы дособрать нужные данные, выделим ссылки на все «несобранные» дома и пройдёмся по ним отдельно.

Если Шаг 5.1 вернул пустой список, вы можете пропустить весь этот шаг.

In [ ]:
# Шаг 5.1. Выделяем ссылки на дома без информации

all_links_no_info = list (final_df[final_df ['Тип дома']=='Нет информации']['Ссылка'])
print (f'Нужно добрать {len (all_links_no_info)} домов')
all_links_no_info

In [ ]:
# Шаг 5.2. Прописываем функцию для сбора данных по домам без информации

def get_a_df_from_one_page_no_info (link):
    
    ''' Функция принимает на вход ссылку на конкретный дом и возвращает в виде датафрейма всего пять основных параметров:
    1. Ссылка на дом
    2. Признан ли дом аварийным
    3. Состояние дома
    4. Тип дома
    5. Количество квартир
    Остальные поля заполняются как «Нет информации».'''
    
    all_rows = []
    try:
        house = soup (requests.get(link).content)

        all_in_table_left = house.find_all ('table')[0].find_all ('td')
        all_in_table_left = [i.text for i in all_in_table_left if i.text != ' ']
        try:
            needs_repair = all_in_table_left[all_in_table_left.index('Дом признан аварийным ')+1]
        except:
            needs_repair = 'Нет информации'
        try:
            raggedness = all_in_table_left [all_in_table_left.index('Состояние дома ')+1]
        except:
            raggedness = 'Нет информации'
        try:
            house_type = all_in_table_left [all_in_table_left.index('Тип дома ')+1]
        except:
            house_type = 'Нет информации'
        try:
            flat_number = all_in_table_left [all_in_table_left.index('Количество квартир ')+1]
        except:
            flat_number = 'Нет информации'
            

        needed_in_table = [link, needs_repair, raggedness, flat_number, house_type, 'Нет информации', 'Нет информации', 'Нет информации']

    except:
        print ('Nothing to show')
        needed_in_table = ['Нет информации', 'Нет информации', 'Нет информации', 
                           'Нет информации', 'Нет информации', 'Нет информации', 'Нет информации']

    try:
        all_rows.append (needed_in_table)
    except:
        print ('No data')
    first_page = pd.DataFrame (all_rows, columns = ['Ссылка', 'Аварийный', 'Состояние дома', 'Количество квартир', 'Тип дома', 
                                       'Площадь жилых в м2', 'Площадь нежилых в м2', 'Культурное наследие'])
    return first_page

In [ ]:
# Шаг 5.3. Проходимся по ссылкам всех домов без информации и собираем данные об их аварийности и типе здания

key = pd.DataFrame (columns = ['Ссылка', 'Аварийный', 'Состояние дома', 'Количество квартир', 'Тип дома', 
                                       'Площадь жилых в м2', 'Площадь нежилых в м2', 'Культурное наследие'])
for i in tqdm (all_links_no_info):
    may = get_a_df_from_one_page_no_info (i)
    key = key.append (may)
    time.sleep (randint (0,2))

winsound.Beep (440, 1000)
key

Оказалось, что почти все дособранные здания обладают одинаковыми характеристиками: неаварийные, исправные многоквартирные дома. Поэтому мы просто заменяем значения в исходном датафрейме.

In [ ]:
# Шаг 5.4. Заменяем значения

final_df.loc[final_df['Ссылка'].isin(all_links_no_info), 'Аварийный'] = 'Нет'
final_df.loc[final_df['Ссылка'].isin(all_links_no_info), 'Состояние дома'] = 'Исправный'
final_df.loc[final_df['Ссылка'].isin(all_links_no_info), 'Тип дома'] = 'Многоквартирный дом'

Ещё раз проверяем датафрейм.

In [ ]:
# Шаг 5.5. Проверка данных

all_links_no_info_2 = list (final_df[final_df ['Тип дома']=='Нет информации']['Ссылка'])
print (f'Нужно добрать {len (all_links_no_info_2)} домов')
print (all_links_no_info)

final_df

Если какая-то ссылка всё-таки обработалась некорректно, заменяем данные вручную.

In [ ]:
# Шаг 5.6. Заменяем значения для ссылки-исключения

final_df.loc [final_df['Ссылка'] == 'https://dom.mingkh.ru/moskva/moskva/716860','Площадь жилых в м2'] = 5487
final_df.loc [final_df['Ссылка'] == 'https://dom.mingkh.ru/moskva/moskva/716860','Площадь нежилых в м2'] = 2089
final_df.loc [final_df['Ссылка'] == 'https://dom.mingkh.ru/moskva/moskva/716860','Культурное наследие'] = 'Нет'
final_df.loc [final_df['Ссылка'] == 'https://dom.mingkh.ru/moskva/moskva/716860','Количество квартир'] = 108
final_df.loc [final_df['Ссылка'] == 'https://dom.mingkh.ru/moskva/moskva/716860']

# Шаг 6. Собираем координаты с помощью API


Соберём координаты с помощью API МинЖКХ. Несмотря на официальную поддержку, этот метод не гарантирует 100 % геокодирования: координаты не всех домов есть в базе. Так, для г. Кумертау не распознано 11 домов из 358 (3 %), а в г. Белорецке — 35 из 370 (9,5 %). В Москве не распознано 435 домов из 32 727, и минимум 7 распознано неверно, итого ошибка — 1,35 %.

Если в базе нет координат дома, в датасет вносятся координаты [0,0]. Это точка в Атлантическом океане, недалеко от Африки.

Когда парсинг заканчивается, раздаётся звук.

In [ ]:
# Шаг 6.1. Добавляем в наш датасет координаты домов

print (f'Начало работы кода: {time.strftime("%H:%M:%S", time.localtime())}')

headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:109.0) Gecko/20100101 Firefox/118.0",
        "Accept-Language": "ru-RU,ru;q=0.8,en-US;q=0.5,en;q=0.3",
        "X-Requested-With": "XMLHttpRequest",
    }

def get_coordinates (house_id):
    
    ''' Функция принимает на вход уникальный идентификатор дома (int) и возвращает список координат (широта, долгота). '''
    try:
        x = requests.get (f'https://dom.mingkh.ru/api/map/house/{house_id}', headers = headers)
        z = x.json()
        coordinates = z['features'][0]['geometry']['coordinates']
        print ('Есть координаты')
    except:
        print ('Нет координат')
        coordinates = [0,0]
    time.sleep (randint (0,1))
    return (coordinates)

final_df ['Координаты'] = final_df['ID дома'].apply (lambda x: get_coordinates(x))
winsound.Beep (440, 1000)
final_df ['Latitude'] = final_df ['Координаты'].apply (lambda x: x[0])
final_df ['Longitude'] = final_df ['Координаты'].apply (lambda x: x[1])

print (f'Конец работы кода: {time.strftime("%H:%M:%S", time.localtime())}')
print ('Код завершил работу. Проверьте датафрейм')

final_df

# Шаг 7. Подготавливаем адреса для геокодирования в QGIS

Этот шаг нужен, если API МинЖКХ не справился или вы по какой-то причине не хотите им пользоваться.

Nominatim OSM распознаёт адреса только в определённом формате, поэтому перед загрузкой в QGIS нам нужно переформатировать адреса с помощью регулярных выражений.

Так, система не понимает «проезд 1-й Тушинский, 3», но понимает «1-й Тушинский проезд, 3».

Адреса с порядковыми номерами и определениями «Малая(ый)/Большая(ой)» унифицировать сложнее. Здесь нужен более сложный алгоритм  с условием «если не найдёшь в Nominatim этот вариант, переделай его по такой схеме».  

Так, в базе дом.минжкх следующие адреса записаны одинаково:
* ул. 1-я Ватутинская, 5
* ул. 1-я Машиностроения, 4

Однако в базе Nominatim OSM они записаны по-разному:
* 1-я Ватутинская ул., 5
* 1-я ул. Машиностроения, 4 к1

Поэтому текущий алгоритм унификации обречён спасать одни адреса и терять другие, потому нуждается в дальнейшей оптимизации.

In [ ]:
# Шаг 7. Унифицируем адреса

import re
final_df ['Адрес'] = final_df ['Адрес'].str.replace ('корпус ', 'к')
final_df ['Адрес'] = final_df ['Адрес'].str.replace ('строение ', 'с')
final_df ['Адрес'] = final_df ['Адрес'].str.replace ('Строение ', 'с')
final_df ['Адрес'] = final_df ['Адрес'].str.replace ('кв-л', 'квартал')
final_df ['Адрес'] = final_df ['Адрес'].str.replace ('пр-д', 'проезд')
final_df ['Адрес'] = final_df ['Адрес'].replace (r'^п\. .*, ул\. (.*),(.*)', r'\1 ул.,\2', regex=True)
final_df ['Адрес'] = final_df ['Адрес'].replace (r'^п\. (.*?), (.*)', r'\2', regex=True)
final_df ['Адрес'] = final_df ['Адрес'].replace (r'^ул\. (.*),(.*)', r'\1 ул.,\2', regex=True)
final_df ['Адрес'] = final_df ['Адрес'].replace (r'^проезд,?\.? (.*),(.*)', r'\1 проезд,\2', regex=True)
final_df ['Адрес'] = final_df ['Адрес'].replace (r'^пер\.,?\.? (.*),(.*)', r'\1 пер,\2', regex=True)
final_df ['Адрес'] = final_df ['Адрес'].str.replace ('пер\.', 'пер')
final_df ['Адрес'] = final_df ['Адрес'].str.replace ('б-р', 'бульвар')
final_df ['Адрес'] = final_df ['Адрес'].str.replace ('пр-кт', 'проспект')
final_df ['Адрес'] = final_df ['Адрес'].replace (r'^туп\.,?\.? (.*),(.*)', r'\1 туп,\2', regex=True)
final_df ['Адрес'] = final_df ['Адрес'].replace (r'^ш\.,?\.? (.*),(.*)', r'\1 ш,\2', regex=True)

Для пакетного геокодирования вам понадобится современная версия QGIS. У меня стоит 3.30.1-'s-Hertogenbosch. Вам нужно зайти в шестерёнку на панели инструментов и найти «Пакетный геокодер Nominatim».

Тем не менее, даже со всеми этапами препроцессинга, алгоритм не распознал или распознал некорректно 22 % адресов. Это вызвано в том числе тем, что в Советском Союзе улицы называли одинаково, так что «Ирония судьбы» актуальна  даже в геоаналитике. 

# Шаг 8. Скачиваем данные

In [ ]:
# Шаг 6. Скачиваем данные

final_df.to_csv (f'Все дома в городе {city} региона {region}.csv')

Спасибо за внимание! Если есть какие-то вопросы, я всегда доступна:

* Мария Казакова, дата-журналистка
* Telegram @oilunem
* marikasakowa@gmail.com